# Camada Bronze:
---

## Tabela Clientes:

### Objetivo:

Entender a padronização e comportamento dos dados para criar a view clientes_bronze a partir da camada RAW.

### Fonte:

`workspace.default.cliente`

### Destino:

`workspace.olist_bronze.clientes_bronze`

In [0]:
-- Garantindo a existência do SCHEMA para armazenamento:

CREATE SCHEMA IF NOT EXISTS workspace.olist_bronze;

In [0]:
-- Verificando a existência da tabela:
SELECT * FROM workspace.default.cliente;

In [0]:
-- Verificando quantas linhas tem na tabela:
SELECT COUNT(*) AS LINHAS 
FROM workspace.default.cliente;

In [0]:
-- Verificando os tipos de dados:
DESC workspace.default.cliente;

Os tipos de dados estão corretamente definidos, com exceção da coluna `customer_zip_code_prefix`, que está tipificada como `BIGINT`. Considerando que o maior código possui cinco algarismos, esse tipo é desnecessariamente grande e resulta em uso ineficiente de memória. Dessa forma, a coluna pode ser convertida para `INT`, mantendo a semântica dos dados e reduzindo o consumo de memória.

In [0]:
-- Verificando missing values em cada coluna:
SELECT
      SUM(CASE WHEN customer_id IS NULL THEN 1 ELSE 0 END) AS customer_id_null,
      SUM(CASE WHEN customer_unique_id IS NULL THEN 1 ELSE 0 END) AS customer_unique_id_null,
      SUM(CASE WHEN customer_zip_code_prefix IS NULL THEN 1 ELSE 0 END) AS customer_zip_code_prefix_null,
      SUM(CASE WHEN customer_city IS NULL THEN 1 ELSE 0 END) AS customer_city_null,
      SUM(CASE WHEN customer_state IS NULL THEN 1 ELSE 0 END) AS customer_state_null
FROM workspace.default.cliente;

Não há valores nulos em nenhuma coluna.

In [0]:
-- Verificando valores inválidos em cada coluna:
SELECT 
      SUM(CASE WHEN customer_id IS NOT NULL AND NOT customer_id RLIKE '^[A-Za-z0-9]+$' THEN 1 ELSE 0 END) AS customer_id_invalid,

      SUM(CASE WHEN customer_unique_id IS NOT NULL AND NOT customer_unique_id RLIKE '^[A-Za-z0-9]+$' THEN 1 ELSE 0 END) AS customer_unique_id_invalid,

      SUM(CASE WHEN customer_zip_code_prefix IS NOT NULL AND NOT customer_zip_code_prefix RLIKE '^[0-9]+$' THEN 1 ELSE 0 END) AS customer_zip_code_prefix_invalid,

      SUM(CASE WHEN customer_city IS NOT NULL AND NOT customer_city RLIKE '^[a-z ]+$' THEN 1 ELSE 0 END) AS customer_city_invalid,

      SUM(CASE WHEN customer_state IS NOT NULL AND NOT customer_state RLIKE '^[A-Z]+$' THEN 1 ELSE 0 END) AS customer_state_invalid

FROM workspace.default.cliente;

Apenas a coluna `customer_city_invalid` apresenta valores inválidos, será necessário observar quais são esse valores para um melhor entendimento.

In [0]:
SELECT 
      customer_city
FROM workspace.default.cliente
WHERE customer_city NOT RLIKE "^[a-z ]+$";

O erro está relacionado unicamente a um ajuste no critério de filtragem, que passou a considerar cidades com caracteres especiais em sua escrita.

In [0]:
SELECT 
      customer_city
FROM workspace.default.cliente
WHERE customer_city NOT RLIKE "^[a-z0-9 '-]+$";

In [0]:
CREATE OR REPLACE VIEW workspace.olist_bronze.clientes_bronze AS
SELECT *

FROM workspace.default.cliente;